In [157]:
!pip install gensim --user
!pip install textblob
!pip install scikit-learn --user
!pip install sklearn --user

In [165]:
!python --version

Python 3.8.5


In [164]:
!pip install scikit-learn==1.0.1
sklearn.__version__

  Using cached scikit_learn-1.0.1-cp38-cp38-win_amd64.whl (7.2 MB)


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\programdata\\anaconda3\\lib\\site-packages\\scikit_learn-0.23.2.dist-info\\COPYING'

'0.23.2'

  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.23.2
    Uninstalling scikit-learn-0.23.2:



Consider using the `--user` option or check the permissions.



In [155]:
import pandas as pd
import numpy as np
import re

#Timing
from tqdm import tqdm

#Word Embeddings
import nltk
from textblob import TextBlob
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

#Models
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to C:\Users\Cameron
[nltk_data]     Milne\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Cameron
[nltk_data]     Milne\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## EDA

In [122]:
scored_tweets = pd.read_csv('scored_tweets.csv')

In [123]:
# scored_tweets

In [124]:
scored_tweets.columns

Index(['created_at', 'text', 'username', 'screen_name', 'verified',
       'followers_count', 'TOXICITY', 'INSULT', 'PROFANITY', 'THREAT',
       'SEXUALLY_EXPLICIT'],
      dtype='object')

In [125]:
scored_tweets.corr()

,TOXICITY,INSULT,PROFANITY,THREAT,SEXUALLY_EXPLICIT
TOXICITY,1.000000,0.945739,0.921274,0.572428,0.700610
INSULT,0.945739,1.000000,0.827742,0.464412,0.542923
PROFANITY,0.921274,0.827742,1.000000,0.442225,0.758937
THREAT,0.572428,0.464412,0.442225,1.000000,0.425812
SEXUALLY_EXPLICIT,0.700610,0.542923,0.758937,0.425812,1.000000


In [126]:
scored_tweets.head(3)

,created_at,text,username,screen_name,verified,followers_count,TOXICITY,INSULT,PROFANITY,THREAT,SEXUALLY_EXPLICIT
0,created_at,text,username,screen_name,verified,followers_count,0.042518,0.015740,0.037332,0.043029,0.040426
1,Thu Dec 02 15:39:21 +0000 2021,This needs to become a real thing. I wanna see...,GhostFedoraTWEET,GhostyRBLX,False,28,0.552407,0.666591,0.313011,0.431656,0.100750
2,Thu Dec 02 15:39:21 +0000 2021,@ENHYPEN_members love u more jake,aii.laa,jayenaii,False,85,0.084299,0.061555,0.109649,0.069504,0.119767


## New Preprocessing

In [127]:
scored_tweets.dropna(subset=['text'], inplace=True)

In [128]:
def preprocessing(tweets_array):
    
    """
    Take in an array of tweets, and return the processed tweets.
    remove stopwords, usernames, punctuation, and words with length of 1
    """
    processed_array = []
    
    for tweet in tqdm(tweets_array):
        processed_punc = re.sub('[^@a-zA-Z ]', '', tweet)  # remove punctuation
        processed = re.sub('https\S+|@\S+|\b\w{,1}\b', '', processed_punc)  # remove https, usernames, len(word) < 1
        words = word_tokenize(processed)
        tokens_without_sw = [word for word in words if not word in stopwords.words()]
        sentence = ' '.join(tokens_without_sw)
        stemmed_sentence = p.stem_sentence(sentence)
        processed_array.append(stemmed_sentence)
    
    return processed_array

In [129]:
scored_tweets['processed'] = preprocessing(scored_tweets['text'])

100%|██████████| 1962/1962 [06:02<00:00,  5.41it/s]


In [130]:
# from nltk.tokenize import sent_tokenize, word_tokenize

# #tokenizes every tweet
# tokenizer = nltk.RegexpTokenizer(r"\w+")
# scored_tweets['processed'] = scored_tweets['text'].apply(tokenizer.tokenize)
# scored_tweets['processed']

In [131]:
# from nltk.corpus import stopwords 

# #Widely used stopword library
# stop_words = set(stopwords.words('english'))

# #gets rid of numbers
# stop_words.update([str(x) for x in np.arange(10)])

# #apply to tokens
# scored_tweets['processed'] = scored_tweets.processed.apply(lambda x: [i for i in x if not i in stop_words])
# scored_tweets['processed']

In [132]:
# scored_tweets['processed'] = [' '.join(l) for l in scored_tweets['processed']]
# scored_tweets['processed']

In [133]:
#Convert Scores to Round Numbers
scored_tweets['TOXICITY'] = round(scored_tweets['TOXICITY'] * 10, 0)
scored_tweets['PROFANITY'] = round(scored_tweets['PROFANITY'] * 10, 0)
scored_tweets['SEXUALLY_EXPLICIT'] = round(scored_tweets['SEXUALLY_EXPLICIT'] * 10, 0)

In [134]:
scored_tweets.head(5)

,created_at,text,username,screen_name,verified,followers_count,TOXICITY,INSULT,PROFANITY,THREAT,SEXUALLY_EXPLICIT,processed
0,created_at,text,username,screen_name,verified,followers_count,0.0,0.015740,0.0,0.043029,0.0,text
1,Thu Dec 02 15:39:21 +0000 2021,This needs to become a real thing. I wanna see...,GhostFedoraTWEET,GhostyRBLX,False,28,6.0,0.666591,3.0,0.431656,1.0,thi need becom real thing i wan see full us sp...
2,Thu Dec 02 15:39:21 +0000 2021,@ENHYPEN_members love u more jake,aii.laa,jayenaii,False,85,1.0,0.061555,1.0,0.069504,1.0,love jake
3,Thu Dec 02 15:39:21 +0000 2021,@ENHYPEN_members i hope you'll get a good rest...,ً,igpshluv,False,2337,1.0,0.103891,1.0,0.506230,1.0,hope youll get good rest fight tomorrow
4,Thu Dec 02 15:39:21 +0000 2021,@Mal_DuBois It was dragged across on a dinghy...,Gavlar87,gavlar87,False,149,2.0,0.123806,1.0,0.298187,2.0,it drag across dinghi


In [135]:
#Confirms range of values for labels
scored_tweets['TOXICITY'].value_counts()

1.0     993
2.0     370
3.0     128
0.0     126
4.0      82
9.0      69
5.0      63
7.0      49
6.0      46
8.0      31
10.0      5
Name: TOXICITY, dtype: int64

## Models

In [136]:
import pickle

In [137]:
!pip install --upgrade xgboost

Requirement already up-to-date: xgboost in c:\programdata\anaconda3\lib\site-packages (1.5.1)


In [138]:
def split_data(df=scored_tweets, feature='processed', label='TOXICITY'):
    from sklearn.model_selection import train_test_split
    
    train_data, test_data = train_test_split(df, test_size=0.25, random_state=7)
    X_train, X_test = np.asarray(train_data['processed']), np.asarray(test_data['processed'])
    y_train, y_test = np.asarray(train_data[label]), np.asarray(test_data[label])
    
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = split_data(df=scored_tweets, feature='processed', label='TOXICITY')

In [139]:
def TFIDF(X_train, X_test):
    from sklearn.feature_extraction.text import TfidfVectorizer

    vectorizer = TfidfVectorizer()
    train_tfidf = vectorizer.fit_transform(X_train)
    test_tfidf = vectorizer.transform(X_test)
    
    return train_tfidf, test_tfidf, vectorizer

train_tfidf, test_tfidf, vectorizer = TFIDF(X_train, X_test)

In [140]:
pickle.dump(vectorizer, open('PerspectiveAPI_vectorizer.pickle', 'wb'))

In [141]:
def XGBoost_Classifier(feature_category):
    from xgboost import XGBClassifier
    from sklearn.metrics import accuracy_score
    
    X_train, X_test, y_train, y_test = split_data(label=feature_category)
    train_tfidf, test_tfidf, _ = TFIDF(X_train, X_test)
    
    model = XGBClassifier()
    model.fit(train_tfidf, y_train)
    
    y_pred = model.predict(test_tfidf)
    acc = accuracy_score(y_pred, y_test)
    
    return (acc, model)

In [148]:
toxicity_results = XGBoost_Classifier('TOXICITY')
toxicity_results

[20:31:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


(0.47046843177189407,
 XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
               gamma=0, gpu_id=-1, importance_type=None,
               interaction_constraints='', learning_rate=0.300000012,
               max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
               monotone_constraints='()', n_estimators=100, n_jobs=8,
               num_parallel_tree=1, objective='multi:softprob', predictor='auto',
               random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
               subsample=1, tree_method='exact', validate_parameters=1,
               verbosity=None))

In [143]:
pickle.dump(toxicity_results[1], open('toxicity_xgb.pickle', 'wb'))

In [149]:
profanity_results = XGBoost_Classifier('PROFANITY')
profanity_results

[20:31:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


(0.4419551934826884,
 XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
               gamma=0, gpu_id=-1, importance_type=None,
               interaction_constraints='', learning_rate=0.300000012,
               max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
               monotone_constraints='()', n_estimators=100, n_jobs=8,
               num_parallel_tree=1, objective='multi:softprob', predictor='auto',
               random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
               subsample=1, tree_method='exact', validate_parameters=1,
               verbosity=None))

In [145]:
pickle.dump(profanity_results[1], open('profanity_xgb.pickle', 'wb'))

In [150]:
sexually_explicit_results = XGBoost_Classifier('SEXUALLY_EXPLICIT')
sexually_explicit_results

[20:31:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


(0.49287169042769857,
 XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
               gamma=0, gpu_id=-1, importance_type=None,
               interaction_constraints='', learning_rate=0.300000012,
               max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
               monotone_constraints='()', n_estimators=100, n_jobs=8,
               num_parallel_tree=1, objective='multi:softprob', predictor='auto',
               random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
               subsample=1, tree_method='exact', validate_parameters=1,
               verbosity=None))

In [147]:
pickle.dump(sexually_explicit_results[1], open('sexually_explicit_xgb.pickle', 'wb'))